In [ ]:
!git clone https://github.com/kakaobrain/minDALL-E.git

In [ ]:
!pip install -U torchtext==0.10.0


In [ ]:
!conda install torch==1.10 torchvision==0.10.0 torchaudio==0.9.0 cudatoolkit=11.3

In [ ]:
import torch
torch.__version__

In [ ]:
!pip install -r requirements.txt

In [ ]:
!nvidia-smi

In [ ]:
# import torch
# from PIL import Image 

In [ ]:
import os
import sys
import math
import argparse
import clip
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
sys.path.append(os.path.dirname(os.getcwd()))






In [ ]:
from dalle.models import Dalle
from dalle.utils.utils import set_seed, clip_score

# device = 'cuda:0'
device = torch.device('cuda')
device

In [ ]:
torch

In [ ]:
model = Dalle.from_pretrained("minDALL-E/1.3B")
model_clip, preprocess_clip = clip.load("ViT-B/32")

model_clip.to(device=device)
model.to(device=device)

In [ ]:
def sampling(prompt, top_k, softmax_temperature, seed, num_candidates=96, num_samples_for_display=36, folder='',filename=''):
    # Setup
    n_row = int(math.sqrt(num_samples_for_display))
    n_col = int(math.sqrt(num_samples_for_display))
    set_seed(seed)
    
    # Sampling
    images = model.sampling(prompt=prompt,
                            top_k=top_k,
                            top_p=None,
                            softmax_temperature=softmax_temperature,
                            num_candidates=num_candidates,
                            device=device).cpu().numpy()
    images = np.transpose(images, (0, 2, 3, 1))

    # CLIP Re-ranking
    rank = clip_score(prompt=prompt, images=images, model_clip=model_clip, preprocess_clip=preprocess_clip, device=device)
    images = images[rank]
    
    images = images[:num_samples_for_display]
    
    if not os.path.exists(f'images/{folder}/'):
        os.makedirs(f'images/{folder}/')
    # return images
    for i,image in enumerate(images):
        im1 = Image.fromarray(np.uint8((images[i])*255))
        im1.save(f"images/{folder}/{filename}-r{i+1}.jpg")

In [ ]:
# if not os.path.exists('images/test/'):
#     os.makedirs('images/test')

In [ ]:
a = torch.tensor([2], device='cuda:0')
a + torch.tensor([1],device=torch.device('cuda'))

In [ ]:
images = sampling(prompt="a man sleeping with his cat next to him", top_k=196, softmax_temperature=1, seed=15, num_candidates=20, num_samples_for_display=10, folder='onetest',filename='man_cat')

In [ ]:
with open("data/mscoco_male_captions.csv",'r') as f:
    lines = f.readlines()
    for idx,line in enumerate(lines[50:]):
        line = line.strip('\n').strip('.').strip().split(',')
        print(line)
#         break
        sampling(prompt=line[2], top_k=256, softmax_temperature=1, seed=15, num_candidates=5, num_samples_for_display=5, folder='mscoco_male',filename=f"male_{idx+50}")



In [ ]:
with open("data/mscoco_female_captions.csv",'r') as f:
    lines = f.readlines()
    for idx,line in enumerate(lines[1:]):
        line = line.strip('\n').strip('.').strip().split(',')
        print(line)
#         break
        sampling(prompt=line[2], top_k=256, softmax_temperature=1, seed=15, num_candidates=5, num_samples_for_display=5, folder='mscoco_female',filename=f"female_{idx+1}")


In [ ]:
# with open("negated_captions/negated_val_captions.txt",'r') as f:
#     lines = f.readlines()
#     for line in lines:
#         line = line.strip('\n').strip('.').strip()
#         sampling(prompt=line, top_k=196, softmax_temperature=1, seed=15, num_candidates=5, num_samples_for_display=5)



In [ ]:
# im1 = Image.fromarray(np.uint8((images[0])*255))
# # Image.fromarray(np.uint8(numpy_image)).convert('RGB')
# im1.save(f"generated_images/hello-1.jpg")

In [ ]:
sorted(os.listdir('images/mscoco_male/'), key=lambda x: (int(x.split('_')[1].split('-')[0]), int(x.split('_')[1].split('-')[1][1])))

In [ ]:
image = preprocess_clip(Image.open("images/mscoco_male/male_1-r2.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a man", "a woman"]).to(device)

with torch.no_grad():
    image_features = model_clip.encode_image(image)
    text_features = model_clip.encode_text(text)
    
    logits_per_image, logits_per_text = model_clip(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    
print("Label probs:", probs)


In [ ]:
def gender_class(im_path):
    image = preprocess_clip(Image.open(im_path)).unsqueeze(0).to(device)
    text = clip.tokenize(["a man", "a woman"]).to(device)

    with torch.no_grad():
        image_features = model_clip.encode_image(image)
        text_features = model_clip.encode_text(text)

        logits_per_image, logits_per_text = model_clip(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        
        print("Label probs:", probs)
        return probs

In [ ]:
image_paths = sorted(os.listdir('images/mscoco_male/'), key=lambda x: (int(x.split('_')[1].split('-')[0]), int(x.split('_')[1].split('-')[1][1])))
results = []
for im_path in image_paths:
    probs = gender_class(f"images/mscoco_male/{im_path}")
    results.append((im_path,probs))
    

In [ ]:
image_paths = sorted(os.listdir('images/mscoco_female/'), key=lambda x: (int(x.split('_')[1].split('-')[0]), int(x.split('_')[1].split('-')[1][1])))
female_results = []
for im_path in image_paths:
    probs = gender_class(f"images/mscoco_female/{im_path}")
    female_results.append((im_path,probs))
    

In [ ]:
torch.cuda.empty_cache()

In [ ]:
results[0][1][0][0]

In [ ]:
#eval male results
threshold = 0.9
male_labels = []
for row in results:
    name = row[0]
    logits = row[1][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    male_labels.append((name,label))

In [ ]:
#eval male results
threshold = 0.9
female_labels = []
for row in female_results:
    name = row[0]
    logits = row[1][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    female_labels.append((name,label))

In [ ]:
male_labels

In [ ]:
female_labels

In [ ]:
import pandas as pd
male_df = pd.DataFrame(male_labels,columns=['img_name','label'])
female_df = pd.DataFrame(female_labels,columns=['img_name','label'])

In [ ]:
male_df.groupby('label').count()

In [ ]:
female_df.groupby('label').count()

In [ ]:
logits_per_image

In [ ]:
logits_per_text

--- Unknown

In [ ]:
with open("data/mscoco_female_unobvious_captions.csv",'r') as f:
    lines = f.readlines()
    for idx,line in enumerate(lines[1:]):
        line = line.strip('\n').strip('.').strip().split(',')
        print(line)
#         break
        sampling(prompt=line[2], top_k=256, softmax_temperature=1, seed=15, num_candidates=5, num_samples_for_display=5, folder='mscoco_unobvious_female',filename=f"unobvious_female_{idx+1}")


In [ ]:
with open("data/mscoco_male_unobvious_captions.csv",'r') as f:
    lines = f.readlines()
    for idx,line in enumerate(lines[1:]):
        line = line.strip('\n').strip('.').strip().split(',')
        print(line)
#         break
        sampling(prompt=line[2], top_k=256, softmax_temperature=1, seed=15, num_candidates=5, num_samples_for_display=5, folder='mscoco_unobvious_male',filename=f"unobvious_male_{idx+1}")


In [ ]:
image_paths = sorted(os.listdir('images/mscoco_unobvious_male/'), key=lambda x: (int(x.split('_')[2].split('-')[0]), int(x.split('_')[2].split('-')[1][1])))
male_unobv_results = []
for im_path in image_paths:
    probs = gender_class(f"images/mscoco_unobvious_male/{im_path}")
    male_unobv_results.append((im_path,probs))
    

In [ ]:
image_paths = sorted(os.listdir('images/mscoco_unobvious_female/'), key=lambda x: (int(x.split('_')[2].split('-')[0]), int(x.split('_')[2].split('-')[1][1])))
female_unobv_results = []
for im_path in image_paths:
    probs = gender_class(f"images/mscoco_unobvious_female/{im_path}")
    female_unobv_results.append((im_path,probs))

In [ ]:
len(os.listdir('images/mscoco_unobvious_female/')), len(os.listdir('images/mscoco_unobvious_male/'))

In [ ]:
#eval male results
threshold = 0.9
unobvious_male_labels = []
for row in male_unobv_results:
    name = row[0]
    logits = row[1][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    unobvious_male_labels.append((name,label))

In [ ]:
#eval male results
threshold = 0.9
unobvious_female_labels = []
for row in female_unobv_results:
    name = row[0]
    logits = row[1][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    unobvious_female_labels.append((name,label))

In [ ]:
import pandas as pd
male_df = pd.DataFrame(unobvious_male_labels,columns=['img_name','label'])
female_df = pd.DataFrame(unobvious_female_labels,columns=['img_name','label'])

In [ ]:
male_df.groupby('label').count()

In [ ]:
female_df.groupby('label').count()

In [ ]:
male_df.groupby('label').count() / len(male_df)

In [ ]:
female_df.groupby('label').count() / len(female_df)

------ threshold = 0.8

In [ ]:

#eval male results
threshold = 0.8
unobvious_male_labels = []
for row in male_unobv_results:
    name = row[0]
    logits = row[1][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    unobvious_male_labels.append((name,label))

#eval male results
threshold = 0.8
unobvious_female_labels = []
for row in female_unobv_results:
    name = row[0]
    logits = row[1][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    unobvious_female_labels.append((name,label))




In [ ]:
import pandas as pd
male_df = pd.DataFrame(unobvious_male_labels,columns=['img_name','label'])
female_df = pd.DataFrame(unobvious_female_labels,columns=['img_name','label'])

In [ ]:
male_df.groupby('label').count()

In [ ]:
female_df.groupby('label').count()

In [ ]:
male_df.groupby('label').count() / len(male_df)

In [ ]:
female_df.groupby('label').count() / len(female_df)

--------

In [ ]:
%cd

In [ ]:
image_paths = sorted(os.listdir('CogView/male_generations/'))
image_paths

In [ ]:
# processed_paths[0].split('_')[0]

In [ ]:
# image_paths = sorted(os.listdir('CogView/male_generations/'), key=lambda x: (int(x.split('_')[2].split('-')[0]), int(x.split('_')[2].split('-')[1][1])))

processed_paths = []
for path in image_paths:
    if 'concat' not in path:
        processed_paths.append(path)

male_cog_results = []
for im_path in processed_paths:
    probs = gender_class(f"CogView/male_generations/{im_path}")
    text = path.split('_')[0]
    male_cog_results.append((text,im_path,probs))

In [ ]:
image_paths = sorted(os.listdir('CogView/female_generations/'))
processed_paths = []
for path in image_paths:
    if 'concat' not in path:
        processed_paths.append(path)

female_cog_results = []
for im_path in processed_paths:
    probs = gender_class(f"CogView/female_generations/{im_path}")
    text = path.split('_')[0]
    female_cog_results.append((text,im_path,probs))

In [ ]:
len(male_cog_results), len(female_cog_results)

In [ ]:
threshold = 0.9
cog_male_labels = []
for row in male_cog_results:
    text = row[0]
    file_path = row[1]
    logits = row[2][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    cog_male_labels.append((text,file_path,label))

In [ ]:
threshold = 0.9
cog_female_labels = []
for row in female_cog_results:
    text = row[0]
    file_path = row[1]
    logits = row[2][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    cog_female_labels.append((text,file_path,label))

In [ ]:
import pandas as pd
male_df = pd.DataFrame(cog_male_labels,columns=['text','img_name','label'])
female_df = pd.DataFrame(cog_female_labels,columns=['text','img_name','label'])

In [ ]:
male_df.groupby('label').count()

In [ ]:
female_df.groupby('label').count()

In [ ]:
threshold = 0.8
cog_male_labels = []
for row in male_cog_results:
    text = row[0]
    file_path = row[1]
    logits = row[2][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    cog_male_labels.append((text,file_path,label))

In [ ]:
threshold = 0.8
cog_female_labels = []
for row in female_cog_results:
    text = row[0]
    file_path = row[1]
    logits = row[2][0]
    
    if logits[0] >=threshold:
        label = 'male'
    elif logits[1] >= threshold:
        label= 'female'
    else:
        label = 'unknown'
    cog_female_labels.append((text,file_path,label))

In [ ]:
male_df = pd.DataFrame(cog_male_labels,columns=['text','img_name','label'])
female_df = pd.DataFrame(cog_female_labels,columns=['text','img_name','label'])

In [ ]:
male_df.groupby('label').count()

In [ ]:
female_df.groupby('label').count()